In [ ]:
// 1. Periods to analyse
var periods = [
  {label: '2017', year: 2017},
  {label: '2018', year: 2018},
  {label: '2019', year: 2019},
  {label: '2020', year: 2020},
  {label: '2021', year: 2021},
  {label: '2022', year: 2022}
];

// 2. Apply a cloud mask

function maskS2clouds(image) {
  var qa = image.select('QA60');
  var cloudBitMask = 1 << 10;
  var cirrusBitMask = 1 << 11;
  var mask = qa.bitwiseAnd(cloudBitMask).eq(0)
                .and(qa.bitwiseAnd(cirrusBitMask).eq(0));
  return image.updateMask(mask)
              .copyProperties(image, ['system:time_start']);
}


// 3. Defining indices function (NDVI, MSAVI, NDRE, SAVI)

function addIndices(image) {

  var ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI');
  var msavi = image.expression(
    '(2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - RED))) / 2', {
      'NIR': image.select('B8').multiply(0.0001),
      'RED': image.select('B4').multiply(0.0001)
    }).rename('MSAVI');
  var ndre = image.normalizedDifference(['B8', 'B5']).rename('NDRE');
  var savi = image.expression(
    '1.5 * ((NIR - RED) / (NIR + RED + 0.5))', {
      'NIR': image.select('B8').multiply(0.0001),
      'RED': image.select('B4').multiply(0.0001),
}).rename('SAVI');

  return image.addBands([ndvi, msavi, ndre, savi]);
}

var allStats = []; // List to stock all the results

// 4.1 Defining fonction to calculate indices on images in buffers

periods.forEach(function(period) {
  var year = period.year;
  var months = [4, 5, 6, 7];  // from April to July

  var ndviList = [];
  var ndreList = [];
  var msaviList = [];
  var saviList = [];

  months.forEach(function(month) {
    var startDate = ee.Date.fromYMD(year, month, 1);
    var endDate = startDate.advance(1, 'month');

    var s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') // Filtering the collection
      .filterDate(startDate, endDate)
      .filterBounds(buffer500)
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
      .map(maskS2clouds)
      .map(addIndices);

    var monthlyMean = s2.mean();


    ndviList.push(monthlyMean.select('NDVI').rename('NDVI_' + month));
    ndreList.push(monthlyMean.select('NDRE').rename('NDRE_' + month));
    msaviList.push(monthlyMean.select('MSAVI').rename('MSAVI_' + month));
    saviList.push(monthlyMean.select('SAVI').rename('SAVI_' + month));
  });

  var ndviStack = ee.Image.cat(ndviList);
  var ndreStack = ee.Image.cat(ndreList);
  var msaviStack = ee.Image.cat(msaviList);
  var saviStack = ee.Image.cat(saviList);

  // 4.2 Defining Statistics function

  function getStats(imgStack, prefix, year) {
    var bands = imgStack.select([
      prefix + '_4', prefix + '_5', prefix + '_6', prefix + '_7'
    ]);

    var mean = bands.reduce(ee.Reducer.mean()).rename(prefix + '_mean');
    var std = bands.reduce(ee.Reducer.stdDev()).rename(prefix + '_std');
    var min = bands.reduce(ee.Reducer.min()).rename(prefix + '_min');
    var max = bands.reduce(ee.Reducer.max()).rename(prefix + '_max');
    var amplitude = max.subtract(min).rename(prefix + '_amplitude');

    return mean.addBands(std).addBands(amplitude);
  }

  var ndviStats = getStats(ndviStack, 'NDVI');
  var ndreStats = getStats(ndreStack, 'NDRE');
  var msaviStats = getStats(msaviStack, 'MSAVI');
  var saviStats = getStats(saviStack, 'SAVI');

  var finalImage = ndviStats.addBands(ndreStats).addBands(msaviStats).addBands(saviStats);

  var zonalStats = finalImage.reduceRegions({
    collection: buffer500,
    reducer: ee.Reducer.mean(),
    scale: 10,
    crs: 'EPSG:25830'
  });

  zonalStats = zonalStats.map(function(feature) {
    return feature.set('year', year);
  });

  print('Zonal stats for ' + year, zonalStats);

  allStats.push(zonalStats); // to add each year to the list
});


// 5. Export CSV to Google Drive

var allYearsStats = ee.FeatureCollection(allStats).flatten(); // concatenate for exportation
Export.table.toDrive({
  collection: allYearsStats,
  description: 'ZonalStats_2017_2022',
  folder: 'Pastureland',
  fileNamePrefix: 'ZonalStats_2017_2022',
  fileFormat: 'CSV'
});

